In [10]:
import os
import numpy as np

In [3]:
# Constants
KAST_RED_PS  = 0.43   # arcseconds/pixel
H_ALPHA_REST = 6562.8 # Angstroms
HUBBLE_CONST = 69.8   # km/s/Mpc
C            = 3e5    # km/s

In [4]:
# Helper functions
def calc_redshift(obs, rest):
    return (obs - rest) / rest

def calc_velocity(redshift):
    return redshift * C # km/s

def calc_distance(velocity):
    return velocity / HUBBLE_CONST # Mpc

In [5]:
centroids = np.load('centroids_golay_filter_cleaned.npy')

In [6]:
# Calculate redshift of galaxy
halpha_obs = np.mean(centroids[:, 0]) # Angstroms
redshift_ugc11876 = calc_redshift(halpha_obs, H_ALPHA_REST)

# Calculate velocity
velocity_ugc11876 = calc_velocity(redshift_ugc11876) # km/s

# Calculate distance
d = calc_distance(velocity_ugc11876) # Mpc

In [7]:
# Estimate the central pixel
center_pix = np.mean(centroids[:, 1])

In [9]:
# Calculate the raidus and velocity curve
pixel_r = np.abs(centroids[:, 1] - center_pix)
radius  = 1e3 * (np.tan(KAST_RED_PS * (2 * np.pi / 360 / 3600) * pixel_r) * d) # Kpc
velocity = np.abs(calc_velocity(calc_redshift(centroids[:, 0], halpha_obs)))   # km/s

In [11]:
np.save("radius.npy", radius)     # Kpc
np.save("velocity.npy", velocity) # km/s

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))

ax.text(0, 100, 'N = {}'.format(len(velocity)), fontsize=14)

ax.plot(radius, velocity, '.', alpha=0.4)

ax.set_xlabel('Radius from nucleus [kpc]', fontsize=14)
ax.set_ylabel(r'Velocity [kms$^{-1}$]', fontsize=14)
ax.set_title('UGC 11876', fontsize=14)

fig.tight_layout()
fig.savefig(os.path.join(data_folder, 'ugc11876_rotation_curve.pdf'))

plt.show()